In [1]:
from sage.all import *
import random
from sage.sat.converters.polybori import CNFEncoder
from sage.sat.solvers.dimacs import DIMACS
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from pycryptosat import Solver

In [2]:
knownplaintext= [1,1,1,1,0,1,1,1,0,1,0,0,0,0,0,1,1,1,1,0,0,0,1,0,1,1,0,1,1,0,1,1]
knownciphertext=[1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,1,0,1,0,0,1,1,0,0,1,1,0,1,1,1,1,1]
key=            [0,1,0,1,1,1,0,0,1,1,1,0,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,1,0,0,1]

selectedKeybits = [i for i in range(40)]
for i in [11,20,25,26,27,32,35,36]:
    selectedKeybits.remove(i)
num_rounds=528

In [3]:
nextfree=0
maxvars=num_rounds+32+64
vars={}

def getVar(v) -> int:
    global nextfree
    global vars
    global maxvars
    if(v[0]=='L' or v[0]=='k'):
        if(v in vars):
            return vars[v]
        else:
            vars[v]=nextfree
            nextfree+=1
            return vars[v]
    else:
        if(v in vars):
            return vars[v]
        else:
            vars[v]=maxvars
            maxvars+=1
            return vars[v]
    

In [4]:
def encrypt(rounds:int, quadratic:bool, B:BooleanPolynomialRing):
        toAdd=[]
        for i in range(rounds):
            if(quadratic):
                toAdd.append(B.gen(getVar(f'L{i+32}'))+B.gen(getVar(f'k{i%64}'))+B.gen(getVar(f'L{i}'))+B.gen(getVar(f'L{i+16}'))+B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+20}'))+B.gen(getVar(f'b{i}'))+B.gen(getVar(f'L{i+26}'))*B.gen(getVar(f'L{i+20}'))+B.gen(getVar(f'L{i+26}'))*B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'L{i+20}'))*B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'L{i+9}'))*B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'b{i}'))*B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'b{i}'))*B.gen(getVar(f'L{i+20}'))+B.gen(getVar(f'a{i}'))*B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'a{i}'))*B.gen(getVar(f'L{i+20}')))
                
                toAdd.append(B.gen(getVar(f'a{i}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+26}')))
                
                toAdd.append(B.gen(getVar(f'b{i}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+1}')))
            else:
                toAdd.append(B.gen(getVar(f'L{i+32}'))+B.gen(getVar(f'k{i%64}'))+B.gen(getVar(f'L{i}'))+B.gen(getVar(f'L{i+16}'))+B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+20}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'L{i+26}'))*B.gen(getVar(f'L{i+20}'))+B.gen(getVar(f'L{i+26}'))*B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'L{i+20}'))*B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'L{i+9}'))*B.gen(getVar(f'L{i+1}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+1}'))*B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+1}'))*B.gen(getVar(f'L{i+20}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+26}'))*B.gen(getVar(f'L{i+9}'))+B.gen(getVar(f'L{i+31}'))*B.gen(getVar(f'L{i+26}'))*B.gen(getVar(f'L{i+20}')))
        return toAdd

In [5]:
def addPlain(pt:list[int],B:BooleanPolynomialRing):
    toAdd=[]
    for i in range(len(pt)):
        if(pt[i]):
            toAdd.append(1+B.gen(getVar(f'L{i}')))
        else:
            toAdd.append(B.gen(getVar(f'L{i}')))
    return toAdd

In [6]:
def addCipher(ct:list[int],B:BooleanPolynomialRing):
    toAdd=[]
    for i in range(len(ct)):
        if(ct[i]):
            toAdd.append(1+B.gen(getVar(f'L{160+i}')))
        else:
            toAdd.append(B.gen(getVar(f'L{160+i}')))
    return toAdd

In [7]:
def addKey(select:list[int],k:list[int],B:BooleanPolynomialRing):
    toAdd=[]
    for kb in select:
        if(k[kb]):
            toAdd.append(1+B.gen(getVar(f'k{kb}')))
        else:
            toAdd.append(B.gen(getVar(f'k{kb}')))
    return toAdd

In [8]:
def encode(r,fn,ho):
    B=BooleanPolynomialRing(r+32+64+r*2,names='x')
    polynoms=[]
    polynoms+=encrypt(r,(not ho),B)
    polynoms+=addPlain(knownplaintext,B)
    polynoms+=addCipher(knownciphertext,B)
    polynoms+=addKey(selectedKeybits,key,B)
    with open(f'{fn}.anf', 'w') as f:
        for p in polynoms:
            f.write(f"{p}\n")
    with open(f'{fn}_x.cnf', 'w') as f:
        solver = CryptoMiniSat()
        e = CNFEncoder(solver, B,use_xor_clauses=True)
        e(polynoms)
        solver.clauses(f'{fn}_x.cnf')
    with open(f'{fn}.cnf', 'w') as f:
        solver = DIMACS(filename=f'{fn}.cnf')
        e = CNFEncoder(solver, B)
        e(polynoms)
        _ = solver.write()

In [9]:
encode(528,"keeloq_ho_allr",true)
encode(528,"keeloq_q_allr",false)

In [74]:
keys={}

for k, v in vars.items():
    if(k[0]=='k'):
        keys[v]=k

In [79]:
def getSol(sol) -> list[int]:
    sol_vars=[int(x) for x in sol.split()]
    obtainedK=[-1 for _ in range(64)]
    for v in sol_vars:
        if abs(v)-1 in keys:
            if(v>0):
                obtainedK[int(keys[abs(v)-1][1:])]=1
            else:
                obtainedK[int(keys[abs(v)-1][1:])]=0
    return obtainedK